In [ ]:
import os
import whisper
from pydub import AudioSegment
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Inicializa o Tkinter e esconde a janela principal
Tk().withdraw()

# Seleciona o arquivo de áudio
caminho_entrada = askopenfilename(
    title="Selecione o arquivo de áudio",
    filetypes=[("Arquivos de áudio", "*.mp3 *.wav *.m4a *.flac")]
)

if not caminho_entrada:
    print("Nenhum arquivo selecionado. Encerrando o programa.")
    exit()

# Pergunta o nome do arquivo de saída
nome_saida = input("Digite o nome do arquivo de saída (sem extensão): ").strip()
if not nome_saida:
    nome_saida = "transcricao"  # nome padrão caso não digite nada

# Caminho completo do arquivo de saída
caminho_saida_txt = os.path.join(
    os.path.expanduser("~"), "Downloads", f"{nome_saida}.txt"
)

# Parâmetros
duracao_bloco_minutos = 5  # tamanho de cada bloco em minutos

# Carrega o modelo Whisper
modelo = whisper.load_model("medium")

# Carrega o áudio
audio = AudioSegment.from_file(caminho_entrada)
duracao_total = len(audio)  # duração em milissegundos
bloco_ms = duracao_bloco_minutos * 60 * 1000

# Processa em blocos
transcricao_total = ""
for i in range(0, duracao_total, bloco_ms):
    bloco = audio[i:i+bloco_ms]
    arquivo_temp = "temp_audio.wav"
    bloco.export(arquivo_temp, format="wav")
    
    resultado = modelo.transcribe(arquivo_temp)
    transcricao_total += resultado["text"] + "\n"
    
    os.remove(arquivo_temp)

# Salva a transcrição
with open(caminho_saida_txt, "w", encoding="utf-8") as f:
    f.write(transcricao_total)

print(f"Transcrição concluída! Arquivo salvo em: {caminho_saida_txt}")
